In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w7p1/mnist_test.csv
/kaggle/input/2022-ai-w7p1/mnist_train.csv
/kaggle/input/2022-ai-w7p1/submission.csv


In [2]:
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
mninst_train = pd.read_csv("/kaggle/input/2022-ai-w7p1/mnist_train.csv")
mninst_test = pd.read_csv("/kaggle/input/2022-ai-w7p1/mnist_test.csv")
submit = pd.read_csv("/kaggle/input/2022-ai-w7p1/submission.csv")

In [4]:
mninst_test = mninst_test.drop(['Unnamed: 0'],axis=1)

In [5]:
test = torch.FloatTensor(np.array(mninst_test)).cuda()

In [6]:
x_train = mninst_train.drop(['Unnamed: 0','784'],axis=1)
y_train = mninst_train['784']

In [7]:
x_train = torch.FloatTensor(np.array(x_train))
y_train = torch.LongTensor(np.array(y_train))

In [8]:
mninst_train=torch.utils.data.TensorDataset(x_train,y_train)

In [9]:
batch_size = 50
data_loader = torch.utils.data.DataLoader(dataset=mninst_train,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    drop_last=True)

In [10]:
layer1 = torch.nn.Linear(784,400,bias=True)
torch.nn.init.xavier_normal_(layer1.weight)
layer2 = torch.nn.Linear(400,300,bias=True)
torch.nn.init.xavier_normal_(layer2.weight)
layer3 = torch.nn.Linear(300,150,bias=True)
torch.nn.init.xavier_normal_(layer3.weight)
layer4 = torch.nn.Linear(150,75,bias=True)
torch.nn.init.xavier_normal_(layer4.weight)
layer5 = torch.nn.Linear(75,10,bias=True)
torch.nn.init.xavier_normal_(layer5.weight)
relu = torch.nn.ReLU()
model = torch.nn.Sequential(layer1, relu, layer2, relu, layer3, relu, layer4, relu, layer5).cuda()

In [11]:
learning_rate = 0.001
loss = torch.nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [12]:
training_epochs = 30
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost=0
    for X,Y in data_loader:
        X=X.view(-1,28*28).cuda()
        Y=Y.cuda()
        
        optimizer.zero_grad()
        hypothesis=model(X)
        cost=loss(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
print("Learnig finished")

Epoch: 0001 cost = 0.584994197
Epoch: 0002 cost = 0.147939831
Epoch: 0003 cost = 0.118733644
Epoch: 0004 cost = 0.103111185
Epoch: 0005 cost = 0.095385224
Epoch: 0006 cost = 0.083331220
Epoch: 0007 cost = 0.070559286
Epoch: 0008 cost = 0.068920515
Epoch: 0009 cost = 0.056926761
Epoch: 0010 cost = 0.060627591
Epoch: 0011 cost = 0.051845919
Epoch: 0012 cost = 0.047354627
Epoch: 0013 cost = 0.041859370
Epoch: 0014 cost = 0.047473192
Epoch: 0015 cost = 0.038990371
Epoch: 0016 cost = 0.042310562
Epoch: 0017 cost = 0.038258869
Epoch: 0018 cost = 0.038537472
Epoch: 0019 cost = 0.033537094
Epoch: 0020 cost = 0.027354956
Epoch: 0021 cost = 0.031251941
Epoch: 0022 cost = 0.032716122
Epoch: 0023 cost = 0.029833525
Epoch: 0024 cost = 0.030364690
Epoch: 0025 cost = 0.029172108
Epoch: 0026 cost = 0.028132791
Epoch: 0027 cost = 0.031507861
Epoch: 0028 cost = 0.029747421
Epoch: 0029 cost = 0.038269874
Epoch: 0030 cost = 0.021754248
Learnig finished


In [13]:
predict = torch.argmax(model(test),dim=1)
submit.columns

Index(['id', 'Label'], dtype='object')

In [14]:
submit['Label']=predict.cpu()
submission=submit.astype(np.int32)
submission.to_csv("submission.csv",mode='w',header=True,index=False)